In [1]:
"!git clone https://github.com/Ludostck/Projet.git"
path = "./Images/"

In [2]:
IMAGE_SIZE = 128


In [3]:
import glob
import PIL
from PIL import Image
import numpy as np
import os
import shutil
from scipy.io import loadmat

def load_data(data_path, classes, dataset='train', image_size=64):
    num_images = 0
    for class_name in classes:
        dir_path = os.path.join(data_path, dataset, class_name)
        num_images += len([item for item in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, item))])
    
    x = np.zeros((num_images, image_size, image_size, 3), dtype=np.uint8)
    y = np.zeros((num_images, 1), dtype=int)
    
    current_index = 0
    
    for idx_class, class_name in enumerate(classes):
        dir_path = os.path.join(data_path, dataset, class_name)
        for item in sorted(os.listdir(dir_path)):
            file_path = os.path.join(dir_path, item)
            if os.path.isfile(file_path):
                with Image.open(file_path) as img:
                    img = img.convert('RGB').resize((image_size, image_size), Image.Resampling.LANCZOS)
                    x[current_index] = np.asarray(img, dtype=np.uint8)
                    y[current_index] = idx_class
                    current_index += 1
                
    return x, y

CLASSES = ['Nébuleuses']
x_train, y_train = load_data(path, CLASSES, dataset='train', image_size=IMAGE_SIZE)

x_val, y_val = load_data(path, CLASSES, dataset='validation', image_size=IMAGE_SIZE)


x_test, y_test = load_data(path, CLASSES, dataset='test', image_size=IMAGE_SIZE)

# Normalisation des entrées via une division par 255 des valeurs de pixel.


x_combined = np.concatenate((x_train, x_val, x_test), axis=0)





In [7]:
# Importation des bibliothèques nécessaires
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, LeakyReLU, BatchNormalization, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

# Chargement et prétraitement des données MNIST
img_rows, img_cols, channels = IMAGE_SIZE, IMAGE_SIZE, 3  # Assurez-vous que IMAGE_SIZE est défini correctement, par exemple, 128

# Supposons que vos images sont déjà dans x_combined et sont de taille 128x128x3
x_combined = x_combined.astype('float32')
x_combined = (x_combined - 127.5) / 127.5  # Normalisation des images


# Paramètres du GAN
noise_dim = 100
optimizer = Adam(0.0002, 0.5)

# Création du générateur
def build_generator():
    model = Sequential()
    model.add(Dense(256, input_dim=noise_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod((img_rows, img_cols, channels)), activation='tanh'))
    model.add(Reshape((img_rows, img_cols, channels)))
    return model

# Création du discriminateur
def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=(img_rows, img_cols, channels)))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Compilation du discriminateur

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Compilation du générateur
generator = build_generator()
z = Input(shape=(noise_dim,))
img = generator(z)
discriminator.trainable = False  # Très important pour l'entraînement du GAN
valid = discriminator(img)

# Compilation du modèle combiné (GAN)
gan_model = Model(z, valid)
gan_model.compile(loss='binary_crossentropy', optimizer=optimizer)

# Fonction d'entraînement
def train(epochs, batch_size=128, save_interval=50):
    half_batch = batch_size // 2
    for epoch in range(epochs):
        # ---------------------
        #  Entraînement du Discriminateur
        # ---------------------
        idx = np.random.randint(0, x_train.shape[0], half_batch)
        imgs = x_train[idx]
        noise = np.random.normal(0, 1, (half_batch, noise_dim))
        gen_imgs = generator.predict(noise)
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Entraînement du Générateur
        # ---------------------
        noise = np.random.normal(0, 1, (batch_size, noise_dim))
        valid_y = np.array([1] * batch_size)
        g_loss = gan_model.train_on_batch(noise, valid_y)

        # Progression de l'entraînement
        print(f'Époque {epoch + 1}/{epochs}, Reste: {epochs - epoch - 1} époques [D loss: {d_loss[0]}, acc.: {100*d_loss[1]:.2f}%] [G loss: {g_loss}]')

        # Sauvegarde des images générées à intervalles réguliers
        if epoch % save_interval == 0:
            save_imgs(epoch)

# Fonction pour sauvegarder les images générées
def save_imgs(epoch):
    r, c = 5, 5  # Configurez les lignes et colonnes pour les images générées
    noise = np.random.normal(0, 1, (r * c, noise_dim))
    gen_imgs = generator.predict(noise)
    # Ajustez les images en fonction de la normalisation utilisée lors de la formation
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    fig.suptitle(f"Époque: {epoch + 1}")
    fig.savefig(f"generated_mnist_{epoch + 1}.png")
    plt.close()


# Entraînement du GAN
train(epochs=10000, batch_size=32, save_interval=200)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
Époque 1/10000, Reste: 9999 époques [D loss: 21.34443473815918, acc.: 12.50%] [G loss: 0.6583513617515564]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Époque 2/10000, Reste: 9998 époques [D loss: 17.736656188964844, acc.: 10.94%] [G loss: 0.5807290077209473]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Époque 3/10000, Reste: 9997 époques [D loss: 15.534639358520508, acc.: 8.02%] [G loss: 0.48426005244255066]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Époque 4/10000, Reste: 9996 époques [D loss: 15.27692985534668, acc.: 8.37%] [G loss: 0.4053371548652649]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Époque 5/10000, Reste: 9995 époques [D loss: 14.808056831359863, acc.: 9.24%] [G loss: 0.35069000720977783]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Époque 6/10000, Reste: 9994 époques [D loss: 15.625768661499023, acc.: 8.71%] [G loss: 0.30297890305519104]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Époque 7/10000, Reste: 9993 époques [D loss: 15.3

: 